In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from seleniumwire import webdriver as wiredriver
from seleniumwire.utils import decode
import json
import pandas as pd
import requests
import numpy as np
import datetime, time
import os

In [18]:
saved_data = f'{os.getcwd()}\data'

In [2]:
# Run mô phỏng chrome
def browser():
  chrome_options = Options()
  chrome_options.add_argument('--headless') # run Chrome in headless mode (without GUI)
  service = Service('/path/to/chromedriver') # path to chromedriver executable
  driver = wiredriver.Chrome(service=service, options=chrome_options)

  # driver = webdriver.Firefox()#webdriver.Chrome(options=chrome_options)
  return driver
  
# Get data from dict value
def flat_dict(data, columns):
    raw = pd.json_normalize(data[0])[[columns]].set_index([pd.Index(['lv1','lv2','lv3'])]).T.reset_index().drop('index',axis=1)
    return raw

# Get detail infor items
def get_in4_item(shopid, itemid):
  item_url = f"https://shopee.vn/api/v4/item/get?itemid={itemid}&shopid={shopid}"

  payload={
  }
  headers = {
    'authority': 'shopee.vn',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,vi;q=0.8',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'x-api-source': 'pc',
    'x-kl-ajax-request': 'Ajax_Request',
    'x-requested-with': 'XMLHttpRequest',
    'x-shopee-language': 'vi'
  }

  response = requests.request("GET", item_url, headers=headers, data=payload)

  item_raw=response.json()
  product_data=pd.json_normalize(item_raw['data'])
  inf = product_data[['shopid','itemid','brand','price','stock','discount','historical_sold','sold','description','item_rating.rating_star','images','categories','shop_vouchers','global_sold']]
  inf = pd.concat([inf,flat_dict(inf['categories'],'display_name')],axis=1)
  return inf
# Get json data
# def get_json_data(url, api):
#     max_retries = 3
#     chrome_options = Options()
#     chrome_options.add_argument('--headless') # run Chrome in headless mode (without GUI)
#     service = Service('/path/to/chromedriver') # path to chromedriver executable
#     driver = wiredriver.Chrome(service=service, options=chrome_options)
#     # url = 'https://shopee.vn/abc-cat.11036194?facet=100678'
#     # api = 'https://shopee.vn/api/v4/search/search_items?by=relevancy&categoryids=100678&limit=60&match_id=11036194&newest=0&order=desc&page_type=search&scenario=PAGE_CATEGORY&version=2'
#     # Access the URL
#     driver.get(url)
#     wait = WebDriverWait(driver, 10)
#     wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
#     while True:
#         try:
#             response = [ i.response for i in driver.requests if i.url.startswith(api)][0]
#             body = decode(response.body, response.headers.get('Content-Encoding', 'Identify'))
#             decoded_body = body.decode('utf8')
#             json_data = json.loads(decoded_body)
#             break
#         except Exception :
#             max_retries -= 1
#             if max_retries <= 0:
#                 None
#             continue
#     return json_data

def get_json_data(url, api):
    chrome_options = Options()
    chrome_options.add_argument('--headless') # run Chrome in headless mode (without GUI)
    service = Service('/path/to/chromedriver') # path to chromedriver executable
    driver = wiredriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    for request in driver.requests:
        if request.response:
            if request.url.startswith(api):
                response = request.response
                # body = decode(response.body, response.headers.get('Content-Encoding', 'Identify'))
                content_encoding = response.headers.get('Content-Encoding')
                if content_encoding is None:
                    content_encoding = 'identity'
                body = decode(response.body, content_encoding)
                decoded_body = body.decode('utf8')
                json_data = json.loads(decoded_body)
                return json_data

In [3]:
# Lấy thông tin sản phẩm theo từ khóa tìm kiếm
def search_get_items(url = f'https://shopee.vn/search?keyword=b%E1%BB%89m%20youli',page_number= 1):
    driver = browser()
    list_search = pd.DataFrame()
    for i in range(page_number):
        try:
            # url = f'https://shopee.vn/search?keyword=s%E1%BB%AFa%20r%E1%BB%ADa%20m%E1%BA%B7t%20nuskin&page={i}'
            url = f'{url}&page={i}'
            driver.get(url)
            for request in driver.requests:
                if request.response:
                    if request.url.startswith('https://shopee.vn/api/v4/search/search_items?by=relevancy&keyword='):
                        response = request.response
                        body = decode(response.body, response.headers.get('Content-Encoding', 'Identify'))
                        decoded_body = body.decode('utf8')
                        json_data = json.loads(decoded_body)
                        r_list_search = pd.concat(pd.json_normalize(i['item_basic']) for i in json_data['items'])
                        r_list_search['page_number'] = i
                        list_search = pd.concat([list_search,r_list_search],axis=0)
                        
        except:
            pass
    list_search = list_search.reset_index().drop('index',axis=1)

    # In danh sách thông tin sản phẩm
    list_item = list_search[['shopid','itemid','name','brand','page_number']].reset_index().drop('index',axis=1)
    list_item.columns = ['shopid','itemid','f_name','brand','f_page_number']
    # more details
    input = list_item [['shopid','itemid']].drop_duplicates().reset_index().drop('index',axis=1)
    dt = pd.DataFrame()
    for i in range(len(input)):
        shopid = input.iloc[[i]]['shopid'][i]
        itemid = input.iloc[[i]]['itemid'][i]
        try:
            dt = pd.concat([dt,get_in4_item(shopid,itemid)],axis=0)
        except:
            pass
    dt = dt.reset_index().drop('index',axis=1)
    final = list_item.merge(dt, how = 'left', on = ['shopid','itemid'])
    final = final.reset_index().drop('index',axis=1)
    # driver.close()
    return final

In [4]:
# Lấy danh sách sản phẩm phải có mã giảm giá
def search_only_voucher():
    final = search_get_items()
    final['shop_vouchers']= final['shop_vouchers'].fillna(value='')
    final['vouchers']= final['shop_vouchers'].apply(lambda x : [i['voucher_code'] for i in x] if i!= None else '' if len(i[0])<=1 else [])
    final['check'] = final['vouchers'].apply(lambda x: len(x))
    data = final[final['check']>0]
    # data = data.drop_duplicates()
    data['link'] = 'https://shopee.vn/abc-i'
    data['link'] = data['link'].str.cat(data[["shopid", "itemid"]].astype(str), sep=".")
    data = data.reset_index().drop('index',axis=1)
    return data

In [26]:
industry_api = f'https://shopee.vn/api/v4/pages/get_category_tree'
# Lấy dữ liệu theo ngành
def get_industry_data(  url = f'https://shopee.vn/abc-cat.11036194?facet=100678',
                        api = industry_api
                        ):    
    # Lấy danh sách ngành
    # craw data
    json_raw = get_json_data(url,api)
    # transform
    cate_all = pd.json_normalize(json_raw['data']['category_list'])
    cat_details = pd.concat([pd.json_normalize(cate_all.iloc[[i]]['children'][i]) for i in range(len(cate_all))],axis=0)
    cat_details = cat_details.reset_index().drop('index',axis=1)[['catid','parent_catid','name','display_name','image']]
    # get final 
    sum_cate = cat_details.merge(cate_all[['catid','name','display_name']], how='left', left_on='parent_catid', right_on='catid')[['catid_x', 'parent_catid', 'name_x', 'display_name_x', 'image', 'name_y', 'display_name_y']]
    sum_cate.columns = ['catid', 'parent_catid', 'name', 'display_name', 'image','parent_name', 'parent_display_name']
    return sum_cate
while True:
    sum_cate = get_industry_data()
    if sum_cate['parent_display_name'].nunique() >= 27:
        break
pd.concat([pd.read_excel(f'{saved_data}\cate_id.xlsx'),sum_cate], axis=0).drop_duplicates().drop_duplicates().reset_index().drop('index', axis=1).to_excel(f'{saved_data}\cate_id.xlsx',index=False)

In [28]:
# Lấy id ngành con của ngành mẹ
# Xác định ngành đã có trong file "ngành con"
file = r'D:\Python\Shopee-Crawler\output data\filter_facet.xlsx'
try:
    already_facet =  list(pd.read_excel(file)['parent_facetid'].unique())
except:
    already_facet = []
dict_sum_cate= list(sum_cate['parent_catid'].unique())
list_search_facet = [i for i in dict_sum_cate if i not in already_facet]
search_facet =  sum_cate[sum_cate['parent_catid'].isin(list_search_facet)][['parent_catid','parent_display_name']].drop_duplicates().to_dict('records')
def get_facet():
    sub_facet = pd.DataFrame()
    for i in range(len(search_facet)):
        while True:
            try:
                cate = search_facet[i]['parent_catid']
                sub_cat_url = f'https://shopee.vn/abc-cat.{cate}'
                sub_cat_api= f'https://shopee.vn/api/v4/search/search_filter_config?match_id={cate}'
                r_sub_cat = get_json_data(sub_cat_url,sub_cat_api)
                raw_data = pd.json_normalize(r_sub_cat['data']['filter_configuration']['dynamic_filter_group_data']['facets'])
            except:
                pass
            if len(raw_data)>1:
                break
        list_sub_facet = raw_data[['catid','category.display_name']].rename(columns={'catid':'facetid','category.display_name': 'facet_name'}).drop_duplicates().reset_index().drop('index',axis=1)
        list_sub_facet['parent_facetid'] = cate
        list_sub_facet['parent_name'] = search_facet[i]['parent_display_name']
        sub_facet = pd.concat([sub_facet, list_sub_facet], axis=0)
        pd.concat([pd.read_excel(file).drop_duplicates(),sub_facet],axis=0).reset_index().drop('index',axis=1).to_excel(file,index=False)
    return sub_facet
sub_facet = get_facet()
# while True:
#     sub_facet = get_facet()
#     if sub_facet['parent_name'].nunique()>=2:
#         break
# sub_facet = sub_facet.reset_index().drop('index',axis=1)
# sub_facet.to_excel(r'D:\Python\Shopee-Crawler\output data\filter_facet.xlsx',index=False)

In [40]:
file = r'D:\Python\Shopee-Crawler\output data\filter_facet.xlsx'
pd.read_excel(file)#['parent_facetid'].unique()

,facetid,facet_name,parent_facetid,parent_name
0,100244,Áo thun,11035567,Thời Trang Nam
1,100050,Áo khoác,11035567,Thời Trang Nam
2,100242,Áo sơ mi,11035567,Thời Trang Nam
3,100048,Hoodie & Áo nỉ,11035567,Thời Trang Nam
4,100009,Phụ Kiện Thời Trang,11035567,Thời Trang Nam
...,...,...,...,...
488,100720,Bảo hộ gia đình,11116484,Dụng cụ và thiết bị tiện ích
489,101186,Keo & chất kết chính công nghiệp,11116484,Dụng cụ và thiết bị tiện ích
490,101798,Kìm,11116484,Dụng cụ và thiết bị tiện ích
491,101809,Dụng cụ hàn điện,11116484,Dụng cụ và thiết bị tiện ích


In [160]:
# Lấy danh sách chứa tên sản phẩm Mẹ bé và Mỹ phẩm từ danh sách menu và filter
selected_industry = sub_facet[sub_facet['parent_name'].isin(['Mẹ & Bé','Sắc Đẹp'])]
dict_menu= selected_industry.to_dict('records')
list_item_by_menu = pd.DataFrame()
for i in range(len(dict_menu)):
    for page_number in range(4): #Lấy 4 trang đầu tiên tương ứng 240 sản phẩm
        try:
            parent_facetid = dict_menu[i]['parent_facetid']
            facetid = dict_menu[i]['facetid']
            menu_url = f'https://shopee.vn/abc-cat.{parent_facetid}?facet={facetid}&page={page_number}'
            menu_api = f'https://shopee.vn/api/v4/search/search_items?'
            raw = get_json_data(menu_url,menu_api)
            list_name = pd.json_normalize(raw['items'])[['item_basic.name']]
            list_name['parent_name'] = dict_menu[i]['parent_name']
            list_name['facet_name'] = dict_menu[i]['facet_name']
            list_item_by_menu = pd.concat([list_item_by_menu, list_name], axis=0)
        except:
            pass
list_item_by_menu = list_item_by_menu.reset_index().drop('index',axis=1)
list_item_by_menu.to_excel(r'D:\Python\Shopee-Crawler\output data\name_item.xlsx',index=False)

In [ ]:
# Lấy danh sách chứa full thông tin sản phẩm từ danh sách menu và filter
selected_industry = sub_facet
dict_menu= selected_industry.to_dict('records')
list_item_by_menu = pd.DataFrame()
for i in range(len(dict_menu)):
    for page_number in range(4): #Lấy 4 trang đầu tiên tương ứng 240 sản phẩm
        try:
            parent_facetid = dict_menu[i]['parent_facetid']
            facetid = dict_menu[i]['facetid']
            menu_url = f'https://shopee.vn/abc-cat.{parent_facetid}?facet={facetid}&page={page_number}'
            menu_api = f'https://shopee.vn/api/v4/search/search_items?'
            raw = get_json_data(menu_url,menu_api)
            list_name = pd.json_normalize(raw['items'])[['item_basic.name']]
            list_name['parent_name'] = dict_menu[i]['parent_name']
            list_name['facet_name'] = dict_menu[i]['facet_name']
            list_item_by_menu = pd.concat([list_item_by_menu, list_name], axis=0)
        except:
            pass
list_item_by_menu = list_item_by_menu.reset_index().drop('index',axis=1)
list_item_by_menu.to_excel(r'D:\Python\Shopee-Crawler\output data\full_item.xlsx',index=False)

In [32]:
parent_facetid = 11035825
facetid = 100559
page_number = 2
menu_url = f'https://shopee.vn/abc-cat.{parent_facetid}?facet={facetid}&page={page_number}'
menu_api = f'https://shopee.vn/api/v4/search/search_items?'
raw = get_json_data(menu_url,menu_api)

In [36]:
t = pd.json_normalize(raw['items'])

In [39]:
list(t.columns)

['adsid',
 'campaignid',
 'distance',
 'match_type',
 'ads_keyword',
 'deduction_info',
 'collection_id',
 'display_name',
 'campaign_stock',
 'json_data',
 'itemid',
 'shopid',
 'algo_image',
 'fe_flags',
 'item_type',
 'foody_item',
 'search_item_tracking',
 'bff_item_tracking',
 'personalized_labels',
 'biz_json',
 'creative_image_id',
 'creative_id',
 'creative_id_int',
 'item_basic.itemid',
 'item_basic.shopid',
 'item_basic.name',
 'item_basic.label_ids',
 'item_basic.image',
 'item_basic.images',
 'item_basic.currency',
 'item_basic.stock',
 'item_basic.status',
 'item_basic.ctime',
 'item_basic.sold',
 'item_basic.historical_sold',
 'item_basic.liked',
 'item_basic.liked_count',
 'item_basic.view_count',
 'item_basic.catid',
 'item_basic.brand',
 'item_basic.cmt_count',
 'item_basic.flag',
 'item_basic.cb_option',
 'item_basic.item_status',
 'item_basic.price',
 'item_basic.price_min',
 'item_basic.price_max',
 'item_basic.price_min_before_discount',
 'item_basic.price_max_befo

In [59]:
aa = pd.DataFrame({'Animal': ['Falcon', 'Falcon',
                              'Parrot', 'Parrot'],
                   'Max Speed': [380., 370., 24., 26.]})
grouped = aa.groupby("Animal", group_keys=False)['Max Speed'].apply(lambda x: max(x))
# grouped = aa.groupby(['Animal'])['Max Speed'].values.rename('industry_category_percentage')

result1 = pd.DataFrame(grouped).reset_index()


In [62]:
df = pd.DataFrame({'col1': [1, 2],
                   'col2': [0.5, 0.75]},
                  index=['row1', 'row2'])

In [64]:
df.to_dict('split')

{'index': ['row1', 'row2'],
 'columns': ['col1', 'col2'],
 'data': [[1, 0.5], [2, 0.75]]}